REINFORCEMENT LEARNING (DQN) TUTORIAL
Author: Adam Paszke

This tutorial shows how to use PyTorch to train a Deep Q Learning (DQN) agent on the CartPole-v0 task from the OpenAI Gym.

Task

The agent has to decide between two actions - moving the cart left or right - so that the pole attached to it stays upright. You can find an official leaderboard with various algorithms and visualizations at the Gym website.

cartpole
cartpole

As the agent observes the current state of the environment and chooses an action, the environment transitions to a new state, and also returns a reward that indicates the consequences of the action. In this task, rewards are +1 for every incremental timestep and the environment terminates if the pole falls over too far or the cart moves more then 2.4 units away from center. This means better performing scenarios will run for longer duration, accumulating larger return.

The CartPole task is designed so that the inputs to the agent are 4 real values representing the environment state (position, velocity, etc.). However, neural networks can solve the task purely by looking at the scene, so we’ll use a patch of the screen centered on the cart as an input. Because of this, our results aren’t directly comparable to the ones from the official leaderboard - our task is much harder. Unfortunately this does slow down the training, because we have to render all the frames.

Strictly speaking, we will present the state as the difference between the current screen patch and the previous one. This will allow the agent to take the velocity of the pole into account from one image.

Packages

First, let’s import needed packages. Firstly, we need gym for the environment (Install using pip install gym). We’ll also use the following from PyTorch:

neural networks (torch.nn)
optimization (torch.optim)
automatic differentiation (torch.autograd)
utilities for vision tasks (torchvision - a separate package).

In [1]:
import gym
import math
import random
import numpy as np

from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import matplotlib
import matplotlib.pyplot as plt

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

env = gym.make('CartPole-v1').unwrapped
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LR = 0.001  # NN optimizer learning rate
# if gpu is to be used
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Replay Memory
We’ll be using experience replay memory for training our DQN. It stores the transitions that the agent observes, allowing us to reuse this data later. By sampling from it randomly, the transitions that build up a batch are decorrelated. It has been shown that this greatly stabilizes and improves the DQN training procedure.

For this, we’re going to need two classses:

Transition - a named tuple representing a single transition in our environment. It essentially maps (state, action) pairs to their (next_state, reward) result, with the state being the screen difference image as described later on.
ReplayMemory - a cyclic buffer of bounded size that holds the transitions observed recently. It also implements a .sample() method for selecting a random batch of transitions for training.

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

Now, let’s define our model. But first, let’s quickly recap what a DQN is.

DQN algorithm
Our environment is deterministic, so all equations presented here are also formulated deterministically for the sake of simplicity. In the reinforcement learning literature, they would also contain expectations over stochastic transitions in the environment.

Our aim will be to train a policy that tries to maximize the discounted, cumulative reward R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_tR 
t 
0
​
 
​
 =∑ 
t=t 
0
​
 
∞
​
 γ 
t−t 
0
​
 
 r 
t
​
 , where R_{t_0}R 
t 
0
​
 
​
  is also known as the return. The discount, \gammaγ, should be a constant between 00 and 11 that ensures the sum converges. It makes rewards from the uncertain far future less important for our agent than the ones in the near future that it can be fairly confident about.

The main idea behind Q-learning is that if we had a function Q^*: State \times Action \rightarrow \mathbb{R}Q 
∗
 :State×Action→R, that could tell us what our return would be, if we were to take an action in a given state, then we could easily construct a policy that maximizes our rewards:

\pi^*(s) = \arg\!\max_a \ Q^*(s, a)
π 
∗
 (s)=arg 
a
max
​
  Q 
∗
 (s,a)
However, we don’t know everything about the world, so we don’t have access to Q^*Q 
∗
 . But, since neural networks are universal function approximators, we can simply create one and train it to resemble Q^*Q 
∗
 .

For our training update rule, we’ll use a fact that every QQ function for some policy obeys the Bellman equation:

Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))
Q 
π
 (s,a)=r+γQ 
π
 (s 
′
 ,π(s 
′
 ))
The difference between the two sides of the equality is known as the temporal difference error, \deltaδ:

\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))
δ=Q(s,a)−(r+γ 
a
max
​
 Q(s 
′
 ,a))
To minimise this error, we will use the Huber loss. The Huber loss acts like the mean squared error when the error is small, but like the mean absolute error when the error is large - this makes it more robust to outliers when the estimates of QQ are very noisy. We calculate this over a batch of transitions, BB, sampled from the replay memory:

\mathcal{L} = \frac{1}{|B|}\sum_{(s, a, s', r) \ \in \ B} \mathcal{L}(\delta)
L= 
∣B∣
1
​
  
(s,a,s 
′
 ,r) ∈ B
∑
​
 L(δ)
\text{where} \quad \mathcal{L}(\delta) = \begin{cases} \frac{1}{2}{\delta^2} & \text{for } |\delta| \le 1, \\ |\delta| - \frac{1}{2} & \text{otherwise.} \end{cases}
whereL(δ)={ 
2
1
​
 δ 
2
 
∣δ∣− 
2
1
​
 
​
  
for ∣δ∣≤1,
otherwise.
​
 
Q-network
Our model will be a convolutional neural network that takes in the difference between the current and previous screen patches. It has two outputs, representing Q(s, \mathrm{left})Q(s,left) and Q(s, \mathrm{right})Q(s,right) (where ss is the input to the network). In effect, the network is trying to predict the expected return of taking each action given the current input.

In [3]:
class DQN(nn.Module):

    def __init__(self, outputs):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(4,256)
        self.fc2 = nn.Linear(256,512)
        self.fc3 = nn.Linear(512,32)
        
#         self.bn1 = nn.BatchNorm2d(4)
#         self.bn2 = nn.BatchNorm2d(16)
#         self.bn3 = nn.BatchNorm2d(32)
        
        self.head = nn.Linear(32, outputs)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        value = self.head(x.view(x.size(0), -1))
        return value

Function for getting the location, velocity, angle and angle velocity of the cartpole.

In [4]:
def get_cart_location_and_angles(env):
    x_loc = env.state[0]
    x_dot = env.state[1]
    theta = env.state[2]
    theta_dot = env.state[3]
    return torch.Tensor([[x_loc, x_dot, theta, theta_dot]])

Training
Hyperparameters and utilities
This cell instantiates our model and its optimizer, and defines some utilities:

select_action - will select an action accordingly to an epsilon greedy policy. Simply put, we’ll sometimes use our model for choosing the action, and sometimes we’ll just sample one uniformly. The probability of choosing a random action will start at EPS_START and will decay exponentially towards EPS_END. EPS_DECAY controls the rate of the decay.
plot_durations - a helper for plotting the durations of episodes, along with an average over the last 100 episodes (the measure used in the official evaluations). The plot will be underneath the cell containing the main training loop, and will update after every episode.

In [5]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

# Get number of actions from gym action space
n_actions = env.action_space.n
# params = get_cart_location_and_angles(env)

policy_net = DQN(n_actions).to(device)
target_net = DQN(n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), LR)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

Training loop
Finally, the code for training our model.

Here, you can find an optimize_model function that performs a single step of the optimization. It first samples a batch, concatenates all the tensors into a single one, computes Q(s_t, a_t)Q(s 
t
​
 ,a 
t
​
 ) and V(s_{t+1}) = \max_a Q(s_{t+1}, a)V(s 
t+1
​
 )=max 
a
​
 Q(s 
t+1
​
 ,a), and combines them into our loss. By definition we set V(s) = 0V(s)=0 if ss is a terminal state. We also use a target network to compute V(s_{t+1})V(s 
t+1
​
 ) for added stability. The target network has its weights kept frozen most of the time, but is updated with the policy network’s weights every so often. This is usually a set number of steps but we shall use episodes for simplicity.
 

In [6]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    print(loss)
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

Below, you can find the main training loop. At the beginning we reset the environment and initialize the state Tensor. Then, we sample an action, execute it, observe the next screen and the reward (always 1), and optimize our model once. When the episode ends (our model fails), we restart the loop.

Below, num_episodes is set small. You should download the notebook and run lot more epsiodes, such as 300+ for meaningful duration improvements.

In [7]:
num_episodes = 1500
highest_reward = 0
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    state = torch.tensor([env.state]).float()
    cum_reward = 0
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        cum_reward += reward
        # Observe new state
        next_state = torch.tensor([env.state]).float()

        # Store the transition in memory
        memory.push(state, action, next_state, reward)
        
        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            break
        env.render()
    if cum_reward > highest_reward:
        highest_reward = cum_reward
        print(cum_reward)
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

/tmp/ipykernel_5964/2008440578.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  state = torch.tensor([env.state]).float()


tensor([12.])
tensor([14.])
tensor([22.])
tensor(0.5100, grad_fn=<SmoothL1LossBackward0>)
tensor(0.4213, grad_fn=<SmoothL1LossBackward0>)
tensor(0.3359, grad_fn=<SmoothL1LossBackward0>)
tensor(0.2518, grad_fn=<SmoothL1LossBackward0>)
tensor(0.1699, grad_fn=<SmoothL1LossBackward0>)
tensor(0.1085, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0628, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0482, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0504, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0751, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0877, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0864, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0748, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0602, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0406, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0274, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0213, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0207, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0220, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0216, grad_fn=<SmoothL1LossBackward0>

tensor(0.0089, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0108, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0106, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0081, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0040, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0021, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0028, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0039, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0066, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0048, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0030, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0017, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0010, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0019, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0023, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0030, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0021, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0013, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0007, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0006, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0010, grad_fn=<SmoothL1LossBack

tensor(0.1761, grad_fn=<SmoothL1LossBackward0>)
tensor(0.1207, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0466, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0184, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0415, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0784, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0800, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0436, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0159, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0160, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0331, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0408, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0341, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0177, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0092, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0107, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0221, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0234, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0140, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0065, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0060, grad_fn=<SmoothL1LossBack

tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0002, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.0001, grad_fn=<SmoothL1LossBackward0>)
tensor(0.6296, grad_fn=<SmoothL1LossBack

...

tensor(2165.5737, grad_fn=<SmoothL1LossBackward0>)
tensor(2511.0378, grad_fn=<SmoothL1LossBackward0>)
tensor(2858.9270, grad_fn=<SmoothL1LossBackward0>)
tensor(2036.9219, grad_fn=<SmoothL1LossBackward0>)
tensor(2985.9167, grad_fn=<SmoothL1LossBackward0>)
tensor(2479.5813, grad_fn=<SmoothL1LossBackward0>)
tensor(2437.7859, grad_fn=<SmoothL1LossBackward0>)
tensor(2968.0146, grad_fn=<SmoothL1LossBackward0>)
tensor(2571.2871, grad_fn=<SmoothL1LossBackward0>)
tensor(2594.1206, grad_fn=<SmoothL1LossBackward0>)
tensor(2604.0417, grad_fn=<SmoothL1LossBackward0>)
tensor(2492.1755, grad_fn=<SmoothL1LossBackward0>)
tensor(2507.5967, grad_fn=<SmoothL1LossBackward0>)
tensor(2878.6387, grad_fn=<SmoothL1LossBackward0>)
tensor(2293.2214, grad_fn=<SmoothL1LossBackward0>)
tensor(2951.0791, grad_fn=<SmoothL1LossBackward0>)
tensor(2668.5266, grad_fn=<SmoothL1LossBackward0>)
tensor(2211.8027, grad_fn=<SmoothL1LossBackward0>)
tensor(1556.7122, grad_fn=<SmoothL1LossBackward0>)
tensor(2079.0110, grad_fn=<Smoo

tensor(4953.4863, grad_fn=<SmoothL1LossBackward0>)
tensor(4673.0273, grad_fn=<SmoothL1LossBackward0>)
tensor(4457.6221, grad_fn=<SmoothL1LossBackward0>)
tensor(4052.9932, grad_fn=<SmoothL1LossBackward0>)
tensor(3435.7329, grad_fn=<SmoothL1LossBackward0>)
tensor(3974.7104, grad_fn=<SmoothL1LossBackward0>)
tensor(4103.8145, grad_fn=<SmoothL1LossBackward0>)
tensor(4037.7539, grad_fn=<SmoothL1LossBackward0>)
tensor(3466.3062, grad_fn=<SmoothL1LossBackward0>)
tensor(4114.3950, grad_fn=<SmoothL1LossBackward0>)
tensor(3656.6897, grad_fn=<SmoothL1LossBackward0>)
tensor(4263.8184, grad_fn=<SmoothL1LossBackward0>)
tensor(2949.4355, grad_fn=<SmoothL1LossBackward0>)
tensor(3777.5852, grad_fn=<SmoothL1LossBackward0>)
tensor(3557.6267, grad_fn=<SmoothL1LossBackward0>)
tensor(3655.1719, grad_fn=<SmoothL1LossBackward0>)
tensor(4198.5039, grad_fn=<SmoothL1LossBackward0>)
tensor(3396.1248, grad_fn=<SmoothL1LossBackward0>)
tensor(3631.3333, grad_fn=<SmoothL1LossBackward0>)
tensor(3300.2649, grad_fn=<Smoo

tensor(14220.8740, grad_fn=<SmoothL1LossBackward0>)
tensor(10726.5322, grad_fn=<SmoothL1LossBackward0>)
tensor(10343.9512, grad_fn=<SmoothL1LossBackward0>)
tensor(8398.2178, grad_fn=<SmoothL1LossBackward0>)
tensor(8147.2041, grad_fn=<SmoothL1LossBackward0>)
tensor(7100.1372, grad_fn=<SmoothL1LossBackward0>)
tensor(6643.3042, grad_fn=<SmoothL1LossBackward0>)
tensor(8082.8027, grad_fn=<SmoothL1LossBackward0>)
tensor(6748.6709, grad_fn=<SmoothL1LossBackward0>)
tensor(7772.4639, grad_fn=<SmoothL1LossBackward0>)
tensor(6285.2954, grad_fn=<SmoothL1LossBackward0>)
tensor(7313.0449, grad_fn=<SmoothL1LossBackward0>)
tensor(5364.2783, grad_fn=<SmoothL1LossBackward0>)
tensor(4971.5254, grad_fn=<SmoothL1LossBackward0>)
tensor(5268.1289, grad_fn=<SmoothL1LossBackward0>)
tensor(5238.7070, grad_fn=<SmoothL1LossBackward0>)
tensor(4632.9033, grad_fn=<SmoothL1LossBackward0>)
tensor(5566.9922, grad_fn=<SmoothL1LossBackward0>)
tensor(4314.0273, grad_fn=<SmoothL1LossBackward0>)
tensor(4382.0654, grad_fn=<S

tensor(2605.7996, grad_fn=<SmoothL1LossBackward0>)
tensor(4910.1187, grad_fn=<SmoothL1LossBackward0>)
tensor(30770.2891, grad_fn=<SmoothL1LossBackward0>)
tensor(31627.6523, grad_fn=<SmoothL1LossBackward0>)
tensor(31283.4023, grad_fn=<SmoothL1LossBackward0>)
tensor(29832.7930, grad_fn=<SmoothL1LossBackward0>)
tensor(30644.4219, grad_fn=<SmoothL1LossBackward0>)
tensor(29149.8887, grad_fn=<SmoothL1LossBackward0>)
tensor(27754.3750, grad_fn=<SmoothL1LossBackward0>)
tensor(26985.8418, grad_fn=<SmoothL1LossBackward0>)
tensor(25089.2227, grad_fn=<SmoothL1LossBackward0>)
tensor(24143.2266, grad_fn=<SmoothL1LossBackward0>)
tensor(23573.7773, grad_fn=<SmoothL1LossBackward0>)
tensor(20954.3008, grad_fn=<SmoothL1LossBackward0>)
tensor(18290.9727, grad_fn=<SmoothL1LossBackward0>)
tensor(17999.2168, grad_fn=<SmoothL1LossBackward0>)
tensor(15833.8867, grad_fn=<SmoothL1LossBackward0>)
tensor(13719.7109, grad_fn=<SmoothL1LossBackward0>)
tensor(12260.7471, grad_fn=<SmoothL1LossBackward0>)
tensor(11763.8

tensor(2779.2925, grad_fn=<SmoothL1LossBackward0>)
tensor(4455.2925, grad_fn=<SmoothL1LossBackward0>)
tensor(3262.2590, grad_fn=<SmoothL1LossBackward0>)
tensor(4465.3506, grad_fn=<SmoothL1LossBackward0>)
tensor(3967.9968, grad_fn=<SmoothL1LossBackward0>)
tensor(2606.0701, grad_fn=<SmoothL1LossBackward0>)
tensor(4353.9790, grad_fn=<SmoothL1LossBackward0>)
tensor(3164.1279, grad_fn=<SmoothL1LossBackward0>)
tensor(3801.5720, grad_fn=<SmoothL1LossBackward0>)
tensor(3778.9971, grad_fn=<SmoothL1LossBackward0>)
tensor(3655.3945, grad_fn=<SmoothL1LossBackward0>)
tensor(3912.3628, grad_fn=<SmoothL1LossBackward0>)
tensor(2364.3196, grad_fn=<SmoothL1LossBackward0>)
tensor(4806.2764, grad_fn=<SmoothL1LossBackward0>)
tensor(3510.2051, grad_fn=<SmoothL1LossBackward0>)
tensor(3734.1877, grad_fn=<SmoothL1LossBackward0>)
tensor(2569.5845, grad_fn=<SmoothL1LossBackward0>)
tensor(7411.3950, grad_fn=<SmoothL1LossBackward0>)
tensor(3393.0681, grad_fn=<SmoothL1LossBackward0>)
tensor(3920.9299, grad_fn=<Smoo